## BBC project: process, hints, and recipes

The major challenge of the BBC project is to transform the list of critics and movies into searchable Python lists and/or dictionaries. The most difficult aspect of this project is the first: scraping the page on the BBC and, using beautiful soup and regular expressions, building a data set that will work.

Once you have the data set, you will be in good shape going forward--the goal after that will be to search for interesting patterns (top movies by country/critic/director/year)--this is the conceptual work you need to be thinking about while you struggle through wrangling your data.

So, how do I wrangle this data? That is the central challenge that you'll be dealing with this week. The HTML page on the BBC site (mirrored on my site) poses a number of challenges. While the layout is relatively simple and consistent--the simplicity actually makes it a little bit harder, because there's not that many HTML tags to help you isolate each unit of data--you can use beautiful soup to isolate the line that contains all the information for the critic, and you can isolate each group of top 10 movies as well. You need to, and this is a bit harder, use beautiful soup find the critic--as well as the list of movies that immediately follow them. (Using beautiful soup to do that is challenging--I have instructions on how to figure it out, but if you can't figure it out--just DM me on Slack and I will help you!)

Yes, that is how this process will work--below I have step-by-step instructions so you can try to write the code yourself. Do your best--and if you can't get there, Slack me and I will help  get your code working so you can move on to the next step.


### Getting started: Data Architecture

The central challenge of this project it's figuring out how you are going to set up your table or tables from this long list of critics and movies. What will each row be? What will the columns be and each row? How can you set it up so that you have the most useful table possible. 

Some things to think about: the main categories of analysis that are possible include movie, director, critic, critic's country, year, and whatever else you bring to this. Try to design a schema that will give you a table that you can run solid queries on. 

You will eventually want to bring this into pandas so you want to keep your table simple and structured as possible. Try to think about how you can transform the main source into one large table that can be aggregated and grouped.

### Interpretive Architecture
**REMEMBER: secondary source** Part of the steps this week, is to find a source you can use to get the country of origin for each director. This is something you need to search for on your own--it will be hard for you to find a single page that has a list of every single director. But see what you can find. In the end, you don't have to have a complete database of every single director, but do your best to get as many as you can.

You don't necessarily have to go in the direction of directors' origin. You can certainly try to think of other categories of interpretation that you can join to this initial dataset. This is how you bring your point-of-view to a relatively large data set that seeks to frame the past 15 years of cinema. How can you bring a different point-of-view to this subject? You can certainly narrow your focus to a specific country, the group of countries, or a region. Either way, think about other data that might bring different types of insight to this list.

### Ready to code?

The first thing you need to do is import beautiful soup & requests like we did in the homework, and scrape the page. 

http://floatingmedia.com/columbia/BBC.html

Okay let's begin!

STEP 1:


In [2]:
##Import your libraries: Beautiful soup, requests, and re (For regular expressions)
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [3]:
# read the URL, and put the HTML page into beautiful soup
raw_html = requests.get("http://floatingmedia.com/columbia/BBC.html").content
soup_doc = BeautifulSoup(raw_html, "html.parser")
print(soup_doc.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   BBC - Culture - The 21st Century’s 100 greatest films: Who voted?
  </title>
  <meta content="story, STORY, story, image, the-100-greatest-films-of-the-21st-century, " name="keywords"/>
  <meta content="We polled 177 critics from around the world – here is how they voted." name="description"/>
  <meta content="The 21st Century’s 100 greatest films: Who voted?" property="og:title">
   <meta content="article" property="og:type">
    <meta content="http://www.bbc.com/culture/story/20160819-the-21st-centurys-100-greatest-films-who-voted" property="og:url">
     <meta content="We polled 177 critics from around the world – here is how they voted." property="og:description">
      <meta content="summary_large_image" name="twitter:card"/>
      <meta content="@BBC_Culture" name="twitter:site"/>
      <meta content="The 21st Century’s 100 greatest fil

In [4]:
#Using beautiful soup find the div tag that contains 
#the entire list of critics and movies
#Make a variable (like all_info) that holds all that information 
all_info = soup_doc.find(class_ = 'body-content')

**STEP 2** Here is where it begins to get tricky: obviously at this point everything we want is surrounded in `<p>` tags. Use a beautiful soup find_all to get a list of every thing in `<p>` tag. Make a variable that contains that list (you could call it all_p or something)


In [5]:
#find_all
all_p = all_info.find_all('p')
all_p

[<p>Communicating with 177 film critics is a time-consuming process. But for every critic who participated – and many more were invited – it wasn’t just a matter of lending their expertise; it was about sharing their passion. The critics who participated hail from 36 countries: 81 from the US, 19 from the UK, five each from Canada, Cuba, France, and Germany, and four each from Australia, Colombia, India, Israel and Italy. Lebanon, the UAE, China, Bangladesh, Chile, Namibia, Kazakhstan and many others are represented too. Of the 177 critics, 55 are women and 122 are men. We present their votes here in alphabetical order.</p>,
 <p><strong>Simon Abrams – Freelance film critic (US)</strong></p>,
 <p>1. Mulholland Drive (David Lynch, 2001)<br/>2. In the Mood for Love (Wong Kar-wai, 2000)<br/>3. The Tree of Life (Terrence Malick, 2011)<br/>4. Yi Yi: A One and a Two (Edward Yang, 2000)<br/>5. Goodbye to Language (Jean-Luc Godard, 2014)<br/>6. The White Meadows (Mohammad Rasoulof, 2009)<br/>7.

**STEP THREE** This is where all the magic has to happen: you need to find a way to loop through all of the `<p>` elements (loop through the list you just got from the find_all()) and pullout critics, and list of movies. 

Critics should not be too hard--every critic entry is embedded in `<strong>` tags. But in order to get the movies attached to that critic--you need to find the `<p>` tag immediately following each `<p><strong>` -- you can do this using next_sibling.

So, you need to build a loop that searches to your `all_p` list:

if it has a `<strong>` tag then 
critic_info = p_line.strong.string
movie_info = p_line.next_sibling

As you go through this loop print(critic_info, movie_info) and see what comes out. If you're getting the critic string followed by movie line's HTML--you've got it!

I give you the beginning of the loop below, and then you can build it piece by piece. If you want to see the overall architecture of the final loop, I have a commented example at the end of the page--it might not be helpful to look at at this point. See how you do step-by-step and if you get stuck at a step Slack me with your code!



In [18]:
##Write your loop for STEP 3 here
#I started this for you,
#Because you only want it to search starting with each critic
#   if line.strong is not None: does that for you
critic_info = []
movie_info = []
for line in all_p[:-3]:
    if line.strong is not None:
        critic_info.append(line.strong.string)
        movie_info.append(line.next_sibling)

**STEP 4**
If your loop is successfully isolating those two lines: now it's time to parse each line with regular expressions. This needs to happen inside the loop--for every critic, and then (in STEP 5) for every movie. Here just **focus on getting the critics name, organization, and country.**

Inside the loop--once you have critic_info -- make a regular expression that pulls out the name of the critic--make a variable called critic_name

`critic_name = findall(regex,critic_info)`

Do the same thing for critic_org and critic_cn

As you go print(critic_name) then print(critic_org), etc.--to make sure you're getting the results. It might help, before you do all these regular expressions in a loop, to just grab one critics line and test regular expressions on it--to make sure that you're getting the right thing. I provided a cell below for you to practice your regular expressions before you put them into the loop.

In [26]:
#Practice/Build your regular expressions here
crit_sample = "Arturo Aguilar – Rolling Stone Mexico (Mexico)"
regex_for_name = r"(^[A-Z].*)\s–\s"
regex_for_org = r"\s–\s([A-Z].*)\s[(]"
regex_for_cn = r"[(]([A-Z].*)[)]"
name = re.findall(regex_for_name,crit_sample)
org = re.findall(regex_for_org,crit_sample)
cn = re.findall(regex_for_cn,crit_sample)

['Mexico']

In [29]:
#Take your working loop from step three
#And put it here With the regular expression parsing inside it
for critic in critic_info:
    names = re.findall(regex_for_name,critic)
    orgs = re.findall(regex_for_org,critic)
    cns = re.findall(regex_for_cn,critic)

**STEP 5**
Now you need to get your **movie names**--this is the trickiest part. You want to use the same loop you have been working on, and get the name of each movie along with the critic information.

To do this you need to search the movie_info variable -- which is each movie followed by a `<BR>` tag. I showed you this in class, but I'll just tell you again how to do this. To get a list of everything that is not a `<BR>` tag, use this method:

`each_movie = movie_info.find_all(string=True)`

This will give you a list called `each_movie`. Which will contain a string for each movie. Like this:

`1. Zero Dark Thirty (Kathryn Bigelow, 2012)`

Build a loop inside the main loop, that goes to each movie and prints out each movie.


In [35]:
##TakeYou're working loop And add the find_all for each_movie
#And the inner loop that loops through each_movie
each_movie = []
for movie in movie_info:
    each_movie.append(movie.find_all(string=True))
each_movie

[['1. Mulholland Drive (David Lynch, 2001)',
  '2. In the Mood for Love (Wong Kar-wai, 2000)',
  '3. The Tree of Life (Terrence Malick, 2011)',
  '4. Yi Yi: A One and a Two (Edward Yang, 2000)',
  '5. Goodbye to Language (Jean-Luc Godard, 2014)',
  '6. The White Meadows (Mohammad Rasoulof, 2009)',
  '7. Night Across the Street (Raoul Ruiz, 2012)',
  '8. Certified Copy (Abbas Kiarostami, 2010)',
  '9. Sparrow (Johnnie To, 2008)',
  '10. Fados (Carlos Saura, 2007)'],
 ['1. In the Mood for Love (Wong Kar-wai, 2000)',
  '2. Eternal Sunshine of the Spotless Mind (Michel Gondry, 2004)',
  '3. Syndromes and a Century (Apichatpong Weerasethakul, 2006)',
  '4. Spirited Away (Hayao Miyazaki, 2001)',
  '5. The Act of Killing (Joshua Oppenheimer, 2012)',
  '6. The Grand Budapest Hotel (Wes Anderson, 2014)',
  '7. The New World (Terrence Malick, 2004)',
  '8. Certified Copy (Abbas Kiarostami, 2010)',
  '9. The World (Jia Zhangke, 2004)',
  '10. Elephant (Gus Van Sant, 2003)'],
 ['1. Zero Dark Thirt

Now that you have that loop working, you need to use regular expressions to get out the name of the movie. First practice getting a regular expression that gets you the name of the movie.


In [3]:
#Practice/Build your regular expressions here
movie_sample = "1. Zero Dark Thirty (Kathryn Bigelow, 2012)"
movie_harder = "1. La commune (Paris, 1871) (Peter Watkins, 2000)"
regex_for_mname = r"[(](\w.*)[,]"
movie_name = re.findall(regex_for_mname,movie_harder)
movie_name

['Paris, 1871) (Peter Watkins']

**STEP 6**
You're almost there!!! Now that you have a working regular expression put that in your inner loop to get the movie name.

So now the entire loop should be getting you 13 elements:
-critic_name
-critic_org
-critic_cn

And an inner loop that will run 10 times (for the 10 movies) and give you 10 instances of:
-rank (this is actually optional, but maybe helpful to keep)
-movie_name
-director
-year

Build this loop using print() on the first one or two critic selections. Just to make sure you are pulling out the right data.




In [15]:
#Get that loop working here
all_movie_info = []
for line in all_p[:-3]:
    try:
        if line.strong is not None:
            movie_info = line.next_sibling.find_all(string=True)
            for movie in movie_info:
                one_movie = {}
                one_movie['movie_name'] = re.findall(r"\d[.]\s(.*)\s[(]", movie)[0]
                one_movie['movie_director'] = re.findall(r"[(](\w.*)[,]", movie)[0]
                one_movie['movie_year'] = re.findall(r"[,]\s(\d{4})", movie)[0]
                one_movie['movie_rank'] = re.findall(r"(\d+)[.]", movie)[0]
                critic_info = line.strong.string
                one_movie['critic_name'] = re.findall(r"(^[A-Z].*)\s–\s",critic_info)[0]
                one_movie['critic_org'] = re.findall(r"\s–\s([A-Z].*)\s[(]",critic_info)[0]
                one_movie['critic_cn'] = re.findall(r"[(]([A-Z].*)[)]",critic_info)[0]
                all_movie_info.append(one_movie)    
    except:
        print("Something went wrong!")
all_movie_info 

[{'movie_name': 'Mulholland Drive',
  'movie_director': 'David Lynch',
  'movie_year': '2001',
  'movie_rank': '1',
  'critic_name': 'Simon Abrams',
  'critic_org': 'Freelance film critic',
  'critic_cn': 'US'},
 {'movie_name': 'In the Mood for Love',
  'movie_director': 'Wong Kar-wai',
  'movie_year': '2000',
  'movie_rank': '2',
  'critic_name': 'Simon Abrams',
  'critic_org': 'Freelance film critic',
  'critic_cn': 'US'},
 {'movie_name': 'The Tree of Life',
  'movie_director': 'Terrence Malick',
  'movie_year': '2011',
  'movie_rank': '3',
  'critic_name': 'Simon Abrams',
  'critic_org': 'Freelance film critic',
  'critic_cn': 'US'},
 {'movie_name': 'Yi Yi: A One and a Two',
  'movie_director': 'Edward Yang',
  'movie_year': '2000',
  'movie_rank': '4',
  'critic_name': 'Simon Abrams',
  'critic_org': 'Freelance film critic',
  'critic_cn': 'US'},
 {'movie_name': 'Goodbye to Language',
  'movie_director': 'Jean-Luc Godard',
  'movie_year': '2014',
  'movie_rank': '5',
  'critic_name

**STEP 7**
This is the final step of the hardest part! If you make it all the way to the end of this let me know and we can discuss what to do next. If you've made it just following instructions, you are in great shape for the rest of this project--if not, don't worry! I will get you through by midweek.

The final step is building a list of lists of all this information.

So you need have a loop that gets everything out--but you also need to figure out **how  you want to organize what you're pulling out.** What should a row look like in your table?




In [84]:
#figure out how you're going to collect your clean information
all_movie_info = []
for line in all_p[1:5]:
    if line.strong is not None:
        movie_info = line.next_sibling.find_all(string=True)
        for movie in movie_info:
            critic_info = line.strong.string
            one_movie_info = re.findall(r"(\d+)[.]\s(.*)\s[(]([A-Z].*)[,]\s(\d{4})", movie)
            movie_critic_info = re.findall(r"(^[A-Z].*)\s–\s([A-Z].*)\s[(]([A-Z].*)[)]",critic_info)
            all_movie_info.append(one_movie_info + movie_critic_info)
all_movie_info[0]
#loop through the beautiful soup elements
#and use the regexes you developed above to get each unit of info


#You will want to build a list that gets appended to list_of_what
#Try to figure out how you want to append things
#That is, how you want to organize your data

    

[('1', 'Mulholland Drive', 'David Lynch', '2001'),
 ('Simon Abrams', 'Freelance film critic', 'US')]

In [ ]:
##Take a peek at your final lists of lists
list_of_what

If you made it this far, congratulations!

You can go ahead and try to build the list of movies and/or the list of directors on your own--they will use similar logic, but they will not be nearly as complicated as this one.

In [16]:
import pandas as pd
df = pd.DataFrame(all_movie_info)
df

,movie_name,movie_director,movie_year,movie_rank,critic_name,critic_org,critic_cn
0,Mulholland Drive,David Lynch,2001,1,Simon Abrams,Freelance film critic,US
1,In the Mood for Love,Wong Kar-wai,2000,2,Simon Abrams,Freelance film critic,US
2,The Tree of Life,Terrence Malick,2011,3,Simon Abrams,Freelance film critic,US
3,Yi Yi: A One and a Two,Edward Yang,2000,4,Simon Abrams,Freelance film critic,US
4,Goodbye to Language,Jean-Luc Godard,2014,5,Simon Abrams,Freelance film critic,US
...,...,...,...,...,...,...,...
1765,The Lives of Others,Florian Henckel von Donnersmarck,2006,6,Raymond Zhou,China Daily,China
1766,Still Life,Jia Zhangke,2006,7,Raymond Zhou,China Daily,China
1767,Birdman,Alejandro González Iñárritu,2014,8,Raymond Zhou,China Daily,China
1768,Infernal Affairs,Andrew Lau and Alan Mak,2002,9,Raymond Zhou,China Daily,China


In [19]:
df.to_csv('all_movie_info.csv',index=False)